# 1. 국토부 오픈API 아파트 실거래가 수집

In [8]:
 ! pip install xmltodict  # 국토부 실거래 데이터 xml -> json 변환

In [9]:
import os
import pandas as pd
import folium
import json
import seaborn as sns
import xmltodict
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, quote
from folium.plugins import HeatMap

In [10]:
molit_serviceKey = 'V3pgDw4Na%2BC2XkjtjQzQ1Co6xR9HCnU3pJw4lPUNX4TU3RdZXu7%2FX60NUnj9UOWrbTpukrfudRi1baplWwp7%2BQ%3D%3D'

In [35]:
def molit_public_api(key, rcode, dealdate):
  try:
    # print(rcode, dealdate)
    base = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?"

    try:
      parameters = "serviceKey=%s&" % (key) + urlencode({quote_plus('LAWD_CD'): rcode, quote_plus('DEAL_YMD'): str(dealdate), quote_plus('numOfRows'): str(9999)}, encoding='UTF-8')
    except:
      return None
        
    url = base + parameters
    # print('url', url)

    request = Request(url)
    response = urlopen(request)
    rescode = response.getcode()
    # print(rescode)
    if rescode == 200:
      response_body = response.read().decode('utf-8')
    else:
      print('error code :' + rescode)

    if response_body == None:
      return None

    jsonData = json.loads(json.dumps(xmltodict.parse(response_body)))
    if jsonData['response']['body']['items'] is not None:
      df = pd.DataFrame(jsonData['response']['body']['items']['item'])
      return df
    else:
      return None

  except Exception as e:
    print('exception occur!', str(e))

In [36]:
def get_yearmonth_list(year, month):
    this_yearmonth = date.today()
    init_yearmonth = datetime.strptime(year+month, "%Y%m").date()
    year_months = []
    while init_yearmonth <= this_yearmonth:
        year_months.append(init_yearmonth)
        init_yearmonth = init_yearmonth + relativedelta(months=1)
    return year_months

In [37]:
year_months = get_yearmonth_list(year='2021', month='4')

In [38]:
for year_month in year_months:
  try:
      dirname = '/content/drive/MyDrive/국토부_실거래'
      yearmonth = year_month.strftime("%Y%m") #202102
      
      filename = os.path.join(dirname, '{}.csv'.format(yearmonth))
      if not os.path.isfile(filename):
        df_molit_api = molit_public_api(molit_serviceKey, rcode='36110', dealdate=yearmonth)
        if df_molit_api is not None:
          df_molit_api.to_csv(filename, encoding='utf8', index=False)
        else:
          print(yearmonth, '조회 가능한 데이터가 없어 파일을 생성할 수 없습니다.')
      else:
        print(yearmonth, '이미 파일이 존재합니다.')
  except Exception as e:
    print(str(e))

202104 이미 파일이 존재합니다.
202105 조회 가능한 데이터가 없어 파일을 생성할 수 없습니다.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/국토부_실거래/202002.csv')
df.head()

# 2. 국토부 오픈API 아파트 실거래가 데이터 불러오기

In [ ]:
# start_y, end_y : YEAR 4자리
# start_m, end_m : MONTH 1~2자리
def get_file_list(start_y, start_m, end_y, end_m):
  if len(str(start_m)) == 1:
    start_m = '{0:02d}'.format(start_m)
  if len(str(end_m)) == 1:
    end_m = '{0:02d}'.format(end_m)

  start = int(str(start_y) + str(start_m))
  end = int(str(end_y) + str(end_m))

  print('조회시작:{}, 조회종료:{}'.format(start, end))

  temp_list = []
  sum = 0
  for file in os.walk('/content/drive/MyDrive/국토부_실거래/').__next__()[2]:
    fileName, fileExt = os.path.splitext(file) # 파일명과 파일확장자 구분
    if start <= int(fileName) <= end:
      df = pd.read_csv(os.path.join('/content/drive/MyDrive/국토부_실거래/', file), index_col=False)
      temp_list.append(df)
  
  df = pd.concat(temp_list, axis=0)
  print('조회완료!')
  return df

In [ ]:
df = get_file_list(start_y=2020, start_m=1, end_y=2020, end_m=12)
df.head()

조회시작:202001, 조회종료:202012
조회완료!
